<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_IRDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Setup and Initial Data
# -----------------------------------------------------------------------------
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
portfolio_data = [
    {'position_id': 1, 'bucket': 'EUR', 'curve': 'OIS', 'tenor_str': '6M', 'sensitivity': -13},
    {'position_id': 2, 'bucket': 'EUR', 'curve': 'OIS', 'tenor_str': '3M', 'sensitivity': 31},
    {'position_id': 3, 'bucket': 'EUR', 'curve': 'SOV', 'tenor_str': '30Y', 'sensitivity': -3155},
    {'position_id': 4, 'bucket': 'GBP', 'curve': 'OIS', 'tenor_str': '3M', 'sensitivity': 18404}
]

# --- Regulatory Parameters ---
# Maps tenor strings to their value in years, as per Article 325l.
TENOR_TO_YEARS = {'3M': 0.25, '6M': 0.5, '1Y': 1, '2Y': 2, '3Y': 3, '5Y': 5, '10Y': 10, '15': 15, '20Y': 20, '30Y': 30}

# Defines the regulatory risk weights by tenor, from Article 325ae, Table 3.
RISK_WEIGHTS = {0.25: 0.017, 0.5: 0.017, 1: 0.016, 2: 0.013, 3: 0.012, 5: 0.011, 10: 0.011, 15: 0.011, 20: 0.011, 30: 0.011}

# A list of "most liquid currencies" which receive a lower risk weight, per Article 325ae.
LIQUID_CURRENCIES = ['EUR', 'GBP', 'USD', 'JPY', 'CHF', 'AUD', 'CAD', 'SEK']

# The theta parameter used in the tenor correlation formula, from Article 325af.
THETA = 0.03

# The uniform cross-bucket correlation for GIRR, from Article 325ag.
GAMMA_BC = 0.5

# Create an initial DataFrame
df = pd.DataFrame(portfolio_data)
df['tenor_years'] = df['tenor_str'].map(TENOR_TO_YEARS)

print("--- Cell 1: Setup Complete ---")
print(f"Loaded {len(df)} positions and all regulatory parameters.")


--- Cell 1: Setup Complete ---
Loaded 4 positions and all regulatory parameters.


In [2]:

# Cell 2: Step 1 & 2 - Identify and Net Sensitivities
# -----------------------------------------------------------------------------
# As per Article 325f(5), sensitivities for identical risk factors are netted.
# In this example, each position is a unique risk factor, so no netting is applied.
print("\n--- Cell 2: Step 1 & 2 - Net Sensitivities ---")
print("Result: Net sensitivities are identical to initial sensitivities.")
print(df[['position_id', 'bucket', 'curve', 'tenor_str', 'sensitivity']].to_string(index=False))



--- Cell 2: Step 1 & 2 - Net Sensitivities ---
Result: Net sensitivities are identical to initial sensitivities.
 position_id bucket curve tenor_str  sensitivity
           1    EUR   OIS        6M          -13
           2    EUR   OIS        3M           31
           3    EUR   SOV       30Y        -3155
           4    GBP   OIS        3M        18404


In [3]:

# Cell 3: Step 3 - Calculate Weighted Sensitivities
# -----------------------------------------------------------------------------
# Each net sensitivity is multiplied by its regulatory risk weight (RW_k) from Article 325ae.
# For liquid currencies, the RW is divided by sqrt(2).
def get_risk_weight(row):
    original_weight = RISK_WEIGHTS[row['tenor_years']]
    applied_weight = original_weight / np.sqrt(2) if row['bucket'] in LIQUID_CURRENCIES else original_weight
    return pd.Series([original_weight, applied_weight])

df[['original_rw', 'applied_rw']] = df.apply(get_risk_weight, axis=1)
df['weighted_sensitivity'] = df['sensitivity'] * df['applied_rw']

print("\n--- Cell 3: Step 3 - Weighted Sensitivities ---")
print(df[['position_id', 'sensitivity', 'tenor_str', 'original_rw', 'applied_rw', 'weighted_sensitivity']].round(4).to_string(index=False))



--- Cell 3: Step 3 - Weighted Sensitivities ---
 position_id  sensitivity tenor_str  original_rw  applied_rw  weighted_sensitivity
           1          -13        6M        0.017      0.0120               -0.1563
           2           31        3M        0.017      0.0120                0.3726
           3        -3155       30Y        0.011      0.0078              -24.5401
           4        18404        3M        0.017      0.0120              221.2311


In [4]:
# Cell 4: Step 4 & 5 - Determine Medium Scenario Correlations
# -----------------------------------------------------------------------------
# Per Article 325af, correlation (rho_kl) is calculated for risk factor pairs within the same bucket.
# Per Article 325ag, the cross-bucket correlation (gamma_bc) is set.
print("\n--- Cell 4: Step 4 & 5 - Medium Scenario Correlations ---")
intra_corrs_medium = {}
eur_bucket = df[df['bucket'] == 'EUR'].to_dict('records')
for i in range(len(eur_bucket)):
    for j in range(i + 1, len(eur_bucket)):
        pos1, pos2 = eur_bucket[i], eur_bucket[j]
        rho_tenor = max(np.exp(-THETA * abs(pos1['tenor_years'] - pos2['tenor_years']) / min(pos1['tenor_years'], pos2['tenor_years'])), 0.4)
        rho_kl = rho_tenor if pos1['curve'] == pos2['curve'] else rho_tenor * 0.999
        intra_corrs_medium[(pos1['position_id'], pos2['position_id'])] = rho_kl

cross_corr_medium = GAMMA_BC

print("Intra-Bucket Correlations (EUR):")
for pair, corr in intra_corrs_medium.items():
    print(f"  - Pos {pair[0]} vs Pos {pair[1]}: {corr:.4f}")
print(f"\nCross-Bucket Correlation (EUR vs GBP): {cross_corr_medium:.0%}")



--- Cell 4: Step 4 & 5 - Medium Scenario Correlations ---
Intra-Bucket Correlations (EUR):
  - Pos 1 vs Pos 2: 0.9704
  - Pos 1 vs Pos 3: 0.3996
  - Pos 2 vs Pos 3: 0.3996

Cross-Bucket Correlation (EUR vs GBP): 50%


In [5]:
# Cell 5: Step 6 & 7 - Calculate Capital for Medium Scenario
# -----------------------------------------------------------------------------
# This cell calculates the bucket-specific capital (K_b) and then aggregates them
# to find the total capital for the Medium Scenario.
print("\n--- Cell 5: Step 6 & 7 - Medium Scenario Capital Calculation ---")
capital_results = {}

def calculate_capital_for_scenario(df, intra_corrs, cross_corr, scenario_name):
    bucket_results = {}
    for bucket_name, group in df.groupby('bucket'):
        positions = group.to_dict('records')
        if len(positions) == 1:
            k_b = abs(positions[0]['weighted_sensitivity'])
        else:
            sum_ws_sq = np.sum(group['weighted_sensitivity']**2)
            cross_term = 0
            for i in range(len(positions)):
                for j in range(i + 1, len(positions)):
                    pos1, pos2 = positions[i], positions[j]
                    rho_kl = intra_corrs.get((pos1['position_id'], pos2['position_id']), 0)
                    cross_term += 2 * rho_kl * pos1['weighted_sensitivity'] * pos2['weighted_sensitivity']
            k_b = np.sqrt(max(0, sum_ws_sq + cross_term))
        bucket_results[bucket_name] = {'K_b': k_b, 'S_b': group['weighted_sensitivity'].sum()}

    sum_kb_sq = sum(res['K_b']**2 for res in bucket_results.values())
    cross_bucket_term = 0
    bucket_names = list(bucket_results.keys())
    for i in range(len(bucket_names)):
        for j in range(i + 1, len(bucket_names)):
            s_b = bucket_results[bucket_names[i]]['S_b']
            s_c = bucket_results[bucket_names[j]]['S_b']
            cross_bucket_term += 2 * cross_corr * s_b * s_c
    total_capital = np.sqrt(max(0, sum_kb_sq + cross_bucket_term))
    return total_capital, bucket_results

capital_results['Medium'], bucket_results_med = calculate_capital_for_scenario(df, intra_corrs_medium, cross_corr_medium, 'Medium')
print("Bucket Capital (K_b):")
for bucket, result in bucket_results_med.items():
    print(f"  - {bucket}: {result['K_b']:.2f}")
print(f"\nTotal Capital for Medium Scenario: {capital_results['Medium']:.2f}")


--- Cell 5: Step 6 & 7 - Medium Scenario Capital Calculation ---
Bucket Capital (K_b):
  - EUR: 24.45
  - GBP: 221.23

Total Capital for Medium Scenario: 210.14


In [6]:
# Cell 6: Step 8 & 9 - High/Low Scenarios and Final Charge
# -----------------------------------------------------------------------------
# Per Article 325h, correlations are stressed and the capital is recalculated.
# The final charge is the maximum of the three scenarios.
print("\n--- Cell 6: Step 8 & 9 - Final Charge Determination ---")

def adjust_correlation(medium_corr, scenario):
    if scenario == 'High': return min(medium_corr * 1.25, 1.0)
    if scenario == 'Low': return max(2 * medium_corr - 1.0, 0.75 * medium_corr)
    return medium_corr

# High Scenario
high_intra = {k: adjust_correlation(v, 'High') for k, v in intra_corrs_medium.items()}
high_cross = adjust_correlation(cross_corr_medium, 'High')
capital_results['High'], _ = calculate_capital_for_scenario(df, high_intra, high_cross, 'High')

# Low Scenario
low_intra = {k: adjust_correlation(v, 'Low') for k, v in intra_corrs_medium.items()}
low_cross = adjust_correlation(cross_corr_medium, 'Low')
capital_results['Low'], _ = calculate_capital_for_scenario(df, low_intra, low_cross, 'Low')

# Final Result
summary_df = pd.DataFrame(list(capital_results.items()), columns=['Scenario', 'Capital Requirement'])
winning_scenario = max(capital_results, key=capital_results.get)
final_charge = capital_results[winning_scenario]

print("Scenario Results Summary:")
print(summary_df.round(2).to_string(index=False))
print(f"\nFinal GIRR Delta Charge ({winning_scenario} Scenario): {final_charge:.2f}")


--- Cell 6: Step 8 & 9 - Final Charge Determination ---
Scenario Results Summary:
Scenario  Capital Requirement
  Medium               210.14
    High               206.91
     Low               213.32

Final GIRR Delta Charge (Low Scenario): 213.32
